##### *Credit to full code :* https://github.com/JohannesBorgqvist/Turing_patterns_bud_scars/tree/main


The code below has been used and modified to have 0 holes (where possible so that the code does not break). It had to be slightly modified for it to run on out laptop. (Installation process and common issues I will address later in the dissertation)



### Solve the RD system (of two equations) on the *surface* of a unit sphere

\begin{align*}
\frac{\partial u}{\partial t} &= D_u \frac{\partial^2 u}{\partial x^2} + \gamma(a - u + u^2 v) \\
\frac{\partial v}{\partial t} &= \frac{\partial^2 v}{\partial x^2} + \gamma(b - u^2 v)
\end{align*}

where $u(\mathbf{x},t)$ and $v(\mathbf{x},t)$ are the chemical concentrations, $D_u$ is the diffusion coefficient and $\gamma$, $a=0.2, b=1$ are parameters.

Before running this code, create two folders (Figures and Meshes) located in the same path as this notebook. This will allow all the output to be saved there and stop errors.

In [ ]:
## Import libraries:

import numpy as np
from matplotlib import pyplot as plt
import gmsh 
import sys 
import math
import meshio
from fenics import *
import pandas as pd 
import multiprocessing as mp
import time
from pandas import read_csv

## Other useful things:

# Write less trick
np.fac = np.math.factorial

### Functions for calculating the properties of the Schnakenberg model:


In [ ]:
# Function 1: "calculate_critical_parameters_Schnakenberg"

# The function takes the two parameters a and b of the
# Schnakenberg RD modelas well as the squared wavenumber
# k_squared as input. It returns the criticald diffusion
# parameter dc and the critical wavenumber gamma_c.

def calculate_steady_states_and_critical_parameters_Schnakenberg(a,b,k_squared):

    # Calculate our steady states
    u_0 = a + b
    v_0 = ( (b) / ( (a + b)** 2))

    # Calculate the four partial derivatives of
    # the Jacobian matrix used in the linear
    # stability analysis
    f_u = ( (b - a) / (b + a) )
    f_v = ( (a + b)**2 )
    g_u = ( (-2*b) / (a+b) )
    g_v = -f_v

    # Calculate the critical diffusion value reported
    # in Chaplain 2001. Since the expression is quite
    # messy, we divide it into three parts.
    d_c = ( (f_u*g_v) - (2*f_v*g_u) )
    d_c +=  np.sqrt( d_c**2 - ((f_u**2)*(g_v**2)) )
    d_c = ( (d_c) / (f_u**2) )

    # Using the critical value of the diffusion d_c,
    # we also calculate the critical wave length number
    # gamma_c reported in Chaplain 2001.
    gamma_c = ( ( 2 * d_c * k_squared) / ( (d_c*f_u) + g_v ) )
    
    # Return the parameters 
    return u_0, v_0, d_c, gamma_c


In [ ]:
# Function 2: "compute_minimal_holeradius_for_pattern_disturbance"

# The function takes the Schnakenberg model parameters a, b, d, and
# gamma, as well as the wavenumber k_squared. It computes and
# returns the minimal radius epsilon of a geodesic disk-shaped hole
# that is needed to affect the Turing patterns. The function also
# returns the spectral parameters n_eps and m_eps that corresponds
# to the eigenfunction(s) causing the disturbance

def compute_minimal_holeradius_for_pattern_disturbance(a,b,d,gamma,n_ref,n_largest):
    
    # Compute the lower and upper excited wavenumber bounds
    # OBS! There should be only one eigenvalue in the excitation
    # interval (gL, gM), namely n_ref*(n_ref+1)
    lm = d*(b-a) - (a+b)**3
    gL = gamma*(lm - np.sqrt(lm**2 -4*d*(a+b)**4))/(2*d*(a+b))
    gM = gamma*(lm + np.sqrt(lm**2 -4*d*(a+b)**4))/(2*d*(a+b))

    
    
    
    
##   Code here not relevant to our problem (have 0 holes)-------------------------------------------------------------------
##   (Note to self: delete it and ammend the rest of the code)
    
    
    # Initialize list for epsilon, n, m
    eps_list = [] 
    
    # Go through LOWER unperturbed eigenvalue parameters to compute
    # the minimal epsilon needed to enter the excitation interval
    # For LOWER n's we enter from below, i.e., hit gL
    for n in range(1,n_ref):

        eigv_n = n*(n+1)          # The unperturbed eigenvalue
        m = 0                     # Corresponding and relevant m-value
        Cnm = 4*(2*n+1)/(n*(n+1)) # Coefficient of leading perturbation term     

        epsilon = np.sqrt((gL - eigv_n)/Cnm)
        eps_list.append([n, m, epsilon])
        #print(n, m, epsilon)


        
    # Go through the given unperturbed eigenvalue parameter n_ref to compute
    # the minimal epsilon needed to exit the excitation interval
    # For n_ref we can exit both below and above, i.e., hit gL and gM
    #print("----")

    n = n_ref  
    eigv_n = n*(n+1)          # The unperturbed eigenvalue

    # m = 0, we can only exit above, i.e., hit gM
    m = 0                     # Corresponding and relevant m-value
    Cnm = 4*(2*n+1)/(n*(n+1)) # Coefficient of leading perturbation term       

    epsilon = np.sqrt((gM - eigv_n)/Cnm)
    eps_list.append([n, m, epsilon])
    #print(n, m, epsilon)

    # m > 0, we can only exit below, i.e., hit gL
    # Loop over the corresponding and relevant m-values
    for m in range(1,n+1):

            Cnm_nom = (2*n+1)*np.fac(m+n)
            Cnm_den = (4**m)*np.fac(n-m)*np.fac(m)*np.fac(m-1)
            Cnm = -Cnm_nom/Cnm_den # Coefficient of leading perturbation term

            epsilon = ((gL - eigv_n)/Cnm)**(1/(2*m))
            eps_list.append([n, m, epsilon])
            


    
    # Go through some HIGHER unperturbed eigenvalue parameters to compute
    # the minimal epsilon needed to enter the excitation interval
    # For HIGHER n's we enter from above, i.e., hit gM
    for n in range(n_ref+1,n_largest+1):
        eigv_n = n*(n+1)          # The unperturbed eigenvalue      
        # Loop over the corresponding and relevant m-values
        # (m = 1, ..., n for HIGHER n's)
        for m in range(1,n+1):
            Cnm_nom = (2*n+1)*np.fac(m+n)
            Cnm_den = (4**m)*np.fac(n-m)*np.fac(m)*np.fac(m-1)
            Cnm = -Cnm_nom/Cnm_den # Coefficient of leading perturbation term
            epsilon = ((gM - eigv_n)/Cnm)**(1/(2*m))
            eps_list.append([n, m, epsilon])



    # Compute the minimal epsilon and corresponding spectral parameters        
    #print(eps_list)
    
    eps_min = 3.14 # Initialise with something large.
    eps_max = 0.000000000000000000001 # Initialise with something small
    
    for i in range(0, len(eps_list)):
        #print(eps_list[i])
        if eps_list[i][2] < eps_min:
            eps_min = eps_list[i][2]
            n_min = eps_list[i][0]
            m_min = eps_list[i][1]
        elif eps_list[i][2] > eps_max:
            eps_max = eps_list[i][2]
            n_max = eps_list[i][0]
            m_max = eps_list[i][1]            

    
    # Return the parameters 
    return (eps_min,eps_max), (n_min,n_max), (m_min,m_max)



In [ ]:
# Function 3: "perturbed_eigenvalues_Schnakenberg"

# This function calculates the perturbed eigenvalues of the
# Schnackenberg model. It takes the eigen value pair (n,m)  as well
# as the hole radius epsilon and return the eigenvalue. 

def perturbed_eigenvalue_Schnakenberg(n,m,epsilon):
    
    # Allocate memory for the eigenvalue that will be returned
    eigen_value = n*(n+1)
    # Calculate the linear term depending on the value of m
    if m == 0:
        next_term = ((4*(2*n+1))/(n*(n+1)))*(epsilon**2)
    elif m>0:
        denom = (4**m)*np.math.factorial(m)*np.math.factorial(m-1)*np.math.factorial(n-m)
        c = ((np.math.factorial(m+n))/(denom))
        next_term = -(2*n+1)*c*(epsilon**(2*m))
        
    # Add the next term in the asymptotic expansion
    eigen_value += next_term
    
    # Return the eigenvalue at hand
    return eigen_value


In [ ]:
# Function 4: "check_Turing_conditions_Scnakenberg"

# This function checks whether the provided parameters (a,b,gamma,d)
# satisfies the Turing conditions and in this case it returns the
# upper bound M and the lower bound L in the Turing analysis. 

def check_Turing_conditions_Scnakenberg(a,b,d):
    # Calculate the four partial derivatives of
    # the Jacobian matrix used in the linear
    # stability analysis evaluated at the steady state
    f_u = ( (b - a) / (b + a) )
    f_v = ( (a + b)**2 )
    g_u = ( (-2*b) / (a+b) )
    g_v = -f_v
    
    # Calculate the determinant of the Jacobian matrix evaluated at the steady state
    det_A = (f_u*g_v) - (f_v*g_u)
    
    # Now calculate our four conditions
    cond_1 = (f_u+g_v<0)
    cond_2 = (det_A>0)
    cond_3 = ((d*f_u+g_v)>0)
    cond_4 = (((d*f_u+g_v)**2-(4*d*det_A))>0)
    
    # Now we define the output depending on whether these conditions are met or not
    if cond_1 and cond_2 and cond_3 and cond_4:
        Turing_conditions = True
        discriminant = np.sqrt((d*f_u+g_v)**2-(4*d*det_A))
        L = ((d*f_u+g_v-discriminant)/(2*d))
        M = ((d*f_u+g_v+discriminant)/(2*d))        
    else:
        Turing_conditions = False
        L = 0
        M = 0
        
    # Finally, return these outputs
    return Turing_conditions,L,M


In [ ]:
# Function 5: "calculate_distance_between_bounds"

# This function takes the following inputs
# 1. The parameter a,
# 2. The parameter b,
# 3. The parameter d,
# and it returns the distance M-L where M is the upper bound for
# Turing instability and L is the lower bound. 

def calculate_distance_between_bounds(a,b,d):
    
    # Calculate the critical diffusion
    # Calculate the four partial derivatives of
    # the Jacobian matrix used in the linear
    # stability analysis
    f_u = ( (b - a) / (b + a) )
    f_v = ( (a + b)**2 )
    g_u = ( (-2*b) / (a+b) )
    g_v = -f_v

    # Calculate the critical diffusion value reported
    # in Chaplain 2001. Since the expression is quite
    # messy, we divide it into three parts.
   
    d_c = ( (f_u*g_v) - (2*f_v*g_u) )
    d_c +=  np.sqrt( d_c**2 - ((f_u**2)*(g_v**2)) )
    d_c = ( (d_c) / (f_u**2) )
    
    # Now, we return the output
    if d <= d_c:
        dist = 0
    else:
        dist = np.sqrt(((d*(b-a) - (a+b)**3)**2) - (4*d*((a+b)**4)))
        dist = dist/(d*(a+b))
        
    # Return distance 
    return dist


In [ ]:
# Function 6: "Compute isolated spectral modes"

# This function computes the isolated spectral modes (if any) on
# the interval [nmin, nmax] given by the  parameters gamma, L,
# and M, i.e., it checks which n:s satisfies (4.5) in Chaplain. 

def compute_isolated_spectral_modes(ninterval, gamma, L, M):
    nmin = ninterval[0]
    nmax = ninterval[1]
    for n in range(nmin, nmax):
        k2 = n*(n+1)
        if gamma*L < k2 and k2 < gamma*M:
            print("n = %d is an isolated spectral mode"%int(n))


### Now can run the following script (for Schnakenberg kinetics):

In [ ]:
# Script:"find_parameters_and_plot_perturbed_eigenvalues"

# Description:
# This script checks whether certain parameters satisfy the Turing conditions and
# then it plots the eigenvalues as functions of the hole radius epsilon. We can refer
# to this script as Philip's spaghetti plots. 

# Define the parameter pairs
a = 0.2
b = 1

## Mode of interest:
n = 1
k_squared = n * (n + 1)
u_0, v_0, d_c, gamma_c = calculate_steady_states_and_critical_parameters_Schnakenberg(a, b, k_squared)


d = 18

gamma = gamma_c
# gamma=100

n_largest = 2

eps_tuple, n_tuple, m_tuple = compute_minimal_holeradius_for_pattern_disturbance(a, b, d, gamma, n, n_largest)
eps_max = eps_tuple[1]
n_max = n_tuple[1]
m_max = m_tuple[1]

hole_cylinder_radius = np.sin(eps_max)

# Print the results
print("\n\n==============================================================================================================================\n")
print("\t Testing parameters and plotting perturbed eigenvalues\n")
print("==============================================================================================================================\n")
print("\n\t\tThe parameters are:\t\t(a,b,gamma,d,n)\t=\t(%0.3f,%0.3f,%0.3f,%0.3f,%0.1f)" % (a, b, gamma, d, n))
print("\t\tThe steady states:\t\t(u_0,v_0)\t=\t(%0.4f,%0.4f)" % (u_0, v_0))
print("\t\tThe critical parameters:\t(d_c,gamma_c)\t=\t(%0.4f,%0.4f)" % (d_c, gamma_c))



# Calculate the Turing conditions
Turing_conditions, L, M = check_Turing_conditions_Scnakenberg(a, b, d)
if Turing_conditions:
    print("\n\t\tThe Turing conditions were satisfied! The lower and upper bounds are:")
    print("\t\t\t Upper:\t\t\tgamma M\t\t=\t%0.3f" % (gamma * M))    
    print("\t\t\t Lower:\t\t\tgamma L\t\t=\t%0.3f\n\n" % (gamma * L))
else:
    print("\n\t\tThe Turing conditions were not satisfied.\n\n")

# print("\t\tThe critical radius:\t(n,m,r)\t=\t(%d,%d,%0.3f)" % (n_max, m_max, hole_cylinder_radius))




### Define new functions for generating the meshes:

In [ ]:
# Description:
# This script contains the sole function which generates the spherical meshes with a single
# hole in them. The holes are generated by jamming a cylinder through the sphere and then
# remove the intersection between the two surfaces.

In [ ]:
# Function 1: "generate_spherical_mesh_with_holes"

# The function takes the following inputs:
# 1. The list of start points of the cylinders c0_list,
# 2. The list of end points of the cylinders c1_list,
# 3. The list of radii of the cylinder hole_radii,
# The function saves a mesh in the meshes folder named after
# the number of holes as well as their respective radii.


## When we call the function we always have 0 holes!

def generate_spherical_mesh_with_holes(c0_list,c1_list,hole_radii):
    
    
    # Part 1 out of 7: Initialise Gmsh
    
    gmsh.initialize(sys.argv)
    
    
    # Part 2 out of 7: Add the larger unit sphere
    
    # Create sphere and get its boundary
    v0 = gmsh.model.occ.addSphere(0,0,0, 1)
    gmsh.model.occ.synchronize()
    s0 = gmsh.model.getBoundary([(3, v0)])
    gmsh.model.occ.remove([(3, v0)])
    
    
    # Part 3 out of 7: Add the holes
    
    # Initialise the rest of the sphere
    rest = [s0]
    
    # Loop of the holes and add them
    for hole_index in range(len(hole_radii)):
        # Extract the points on the cylindes
        c0 = c0_list[hole_index]
        c1 = c1_list[hole_index]
        
        # Extract the hole radius
        hole_radius = hole_radii[hole_index]
        if hole_radius > 0:
            
            # Create the cylinder
            v1 = gmsh.model.occ.addCylinder(c0[0],c0[1],c0[2], c1[0],c1[1],c1[2], np.sin(hole_radius))
            # Create the hole being the intersection between the unit sphere and the cylinder
            the_hole = gmsh.model.occ.intersect(rest[0], [(3, v1)], removeObject=False)
            # Remove the hole
            rest = gmsh.model.occ.cut(rest[0], the_hole[0], removeObject=True)    
    
    
    
    # Part 4 out of 7: Finalise the mesh
    
    # fragment all surfaces to make everything conformal
    gmsh.model.occ.synchronize()
    
    # set mesh size
    gmsh.option.setNumber('Mesh.MeshSizeMax', 0.055)
    
    
    
    # Part 5 out of 7: Add physical regions
    
    # Add the rest of the sphere
    rest_of_sphere = gmsh.model.addPhysicalGroup(2,[rest[0][0][1]],1)
    gmsh.model.setPhysicalName(2,rest_of_sphere,"The sphere with a hole with radius, r=" + str(hole_radius))
    
    
    
    # Part 6 out of 7: Add colours to the mesh
    
    # For nice colours, please see (https://colorbrewer2.org/)...
    
    # THE REST OF THE SPHERE
    gmsh.model.setColor([(2,rest[0][0][1])], 2, 56, 88)  # Darkest blue
    
    
    # Part 7 out of 7: Generate the mesh   
    
    # Generate the two dimensional mesh (as we work with surfaces) 
    gmsh.model.mesh.generate(2)
    
    # Allocate a file name for the mesh
    if len(hole_radii) == 1 and hole_radii[0]== 0:
        mesh_name = "Meshes/s_h_0.msh"
    else:
        mesh_name = "Meshes/s_h_" + str(len(hole_radii)) + "_"
        
        # Loop over the hole radii and add these to the file names
        for hole_radius in hole_radii:
            mesh_name += "r_" + str(round(hole_radius,3)).replace(".","p") + "_"
            
        # Add the suffix
        mesh_name += ".msh"
   
    # Write the mesh to our file
    gmsh.write(mesh_name.replace("_.msh",".msh"))   
    
    # Shut down gmsh
    gmsh.finalize()


In [ ]:
# Description:
# This is the script where we automate the generation of the spherical meshes with
# holes in them.  This function calls the only function that is stored in the
# script called "toolbox_generate_meshes.py". 

In [ ]:
# Generate the meshes

# Create the hole radii in an automated fashion
hole_radii_list = [[r] for r in np.arange(0,0.75,0.05)]

# Allocate the lists of points defining the points on the sphere
c0_lists = []
c1_lists = []

# Generate a list of points defining the center endpoints of the hole-making cylinder
# The spherical hole should be centered at south pole (0,0,-1) as in Bandle

for index in range(len(hole_radii_list)):
    c0_lists.append([(0,0,0)])
    c1_lists.append([(0,0,-1)])
    
# Loop over all points and radii to generate the corresponding meshes
for list_index in range(len(hole_radii_list)):
    generate_spherical_mesh_with_holes(c0_lists[list_index],c1_lists[list_index],hole_radii_list[list_index])



In [ ]:
# Function to convert mesh from MSH to XDMF 

def convert_mesh(mesh_name):
    # Read the MSH file
    msh = meshio.read(mesh_name + ".msh")

    # Extract triangle cells from the MSH mesh
    cells = msh.get_cells_type("triangle")
    cell_data = msh.get_cell_data("gmsh:physical", "triangle")

    # Create an XDMF mesh
    xdmf_mesh = meshio.Mesh(points=msh.points, cells={"triangle": cells},
                            cell_data={"name_to_read": [cell_data]})

    # Write the XDMF mesh
    meshio.write(mesh_name + ".xdmf", xdmf_mesh)

# Convert meshes with different radii (changed to only r=0)
for hole_radius in np.arange(0):
    if hole_radius == 0:
        mesh_name = "s_h_0"
    else:
        mesh_name = "s_h_1_r_" + str(round(hole_radius, 3)).replace(".", "p")

    # Convert the mesh
    convert_mesh("Meshes/" + mesh_name)

    print(f"Converted mesh: {mesh_name}")

### Functions for conducting the FEM simulations:


In [ ]:
# Description:
# This is the main script containing all important functions needed in order to
# run the FEM simulations of the Schnakenberg model on the sphere with holes.

# ### NOTE ###

# This script uses the finite difference scheme 1-SBEM in time, resulting in two
# (n x n) systems of equations. The code has been hard coded to be more efficient
# for precisely the scheme 1-SBEM. To play around with and test other types of
# finite difference schemes it's recommended to use the script
# "toolbox_FEM_simulations_Schnakenberg_sphere_with_holes_MixedSystems".

In [ ]:
# Function 1: "read_mesh_Schnakenberg_sphere_with_holes"

# The function takes the number of holes as inputs and
# returns the following four outputs:
# 1. The number of holes in a variable called "num_holes",
# 2. A list of all the radii of the holes called "radii_holes",
# 3. A mesh_function "mf_subdomain",
# 4. A list of integration measures "dx_list" used in the
# variational formulation.

def read_mesh_Schnakenberg_sphere_with_holes(num_holes,radii_holes):
    
    # Allocate memory for the mesh and the mesh value collection
    mesh = Mesh()
    mvc_subdomains = MeshValueCollection("size_t", mesh, 2)
    
    # Define a mesh function taking the subdomains into account
    mf_subdomains = 0
    
    # Allocate memory for a list containing all the integration
    # measures involved in the variational formulation
    dx_list = []    
    # Define the string of the mesh that we want to read
    mesh_str = "Meshes/s_h_" + str(num_holes)
    
    
    # Define the string in which we read the mesh
    # depending on the number of holes
    if num_holes > 0:
        # Loop over the radii and add these to the string name of the mesh
        for radius in radii_holes:
            mesh_str += "_r_" + str(round(radius,3)).replace(".","p") + "_"
            
    # Now, we add the file format to the string as well
    mesh_str += ".xdmf"
    
    # And tidy the file name up in necessary
    mesh_str = mesh_str.replace("_.xdmf",".xdmf")
    print(mesh_str)
    
    # Read in the mesh and the subdomains into these two variables
    with XDMFFile(mesh_str) as infile:
        infile.read(mesh)
        infile.read(mvc_subdomains, "name_to_read")
        
    # Define a mesh function taking the subdomains into account
    mf_subdomains = cpp.mesh.MeshFunctionSizet(mesh, mvc_subdomains)
    
    # Add our integration measure to the list of integration measure
    dx_list.append(Measure("dx", domain=mesh, subdomain_data=mf_subdomains, subdomain_id=1))
    
    # Lastly, return our mesh, the mesh value collection, the mesh
    # function and the integration measures.
    return mesh, mvc_subdomains, mf_subdomains, dx_list


In [ ]:
# Function 2: "set_up_FEM_Schnakenberg_sphere_with_holes"

# The function sets up the FEM framework for the Schnakenberg
# model on the sphere with holes. It takes a mesh as input
# and then it returns the function space H.

def define_Hilbert_space_Schnakenberg_sphere_with_holes(mesh):

    # Define the basis finite elements and their order. We pick linear basis functions. 
    P1 = FiniteElement("P", mesh.ufl_cell(), 1)

    # Define a mixed element since we have a coupled system of two PDEs
    element = MixedElement([P1, P1])

    # Collect everything in the function space (name it H for a Hilbert space)
    H = FunctionSpace(mesh, element)

    # Return all of these elements
    return H


In [ ]:
# Function 3: "perturbations_ic__Schnakenberg_sphere_with_holes"

# The function is a help function for the Function 5 called "initial_conditions_Schnakenberg_sphere_with_holes" which sets the initial conditions of the system. It generates a vector where each element corresponds to a node in the mesh of interest, and each element in this vector corresponds to a perturbation error drawn from a standard normal distribution. The inputs are the vector itself called epsilon and the parameter sigma determining the variance of the standard normal distribution from which the perturbations of the steady states in the initial conditions are drawn.


def perturbations_ic__Schnakenberg_sphere_with_holes(epsilon,sigma):
    
    #epsilon.vector().set_local(np.random.normal(0,sigma,epsilon.vector().local_size()))
    # We use a uniform random distribution between -sigma and sigma
    
    epsilon.vector().set_local(np.random.uniform(-sigma,sigma,epsilon.vector().local_size()))
    return epsilon

In [ ]:
# Function 4: "initial_conditions_Schnakenberg_sphere_with_holes"

# The function sets the initial conditions for the system and writes these two 
# the function u_prev which is given as an input. The initial conditions for 
# the two states in the Schnakenberg are set to a small perturbation around the 
# steady state. The perturbation is given by a normal distribution with zero mean 
# and where the variance is determined by the input sigma. Hence, no output is
# returned and the following input must be provided:

# 1. The Hilbert space H,
# 2. The mesh stored in the variable mesh,
# 3. The mesh function mf_subdomains indicating the subdomains in the mesh,
# 4. The number of holes on the sphere stored in num_holes,
# 5. The steady states of the Schnakenberg model stored in steady_states,
# 6. The variance of the perturbation determined by sigma,
# 7. The FEM functions on the function space H stored in u and v in which we
#    set the initial conditions,
# 8. A logical variable called ICs_around_steady_states which determines 
#    whether the initial conditions are set to the steady states values or not. 


def initial_conditions_Schnakenberg_sphere_with_holes(H,mesh,mf_subdomains,num_holes,steady_states,sigma,u,v,ICs_around_steady_states):
    
    # DETAILED DESCRIPTION OF WHAT THE FUNCTION DOES
    
    # We assign the following initial conditions to the sphere:
    #1. u(t=0,*)= u0 + epsilon
    #2. v(t=0,*)= v0 + epsilon
    # where u0 and v0 are the provided steady states stored in the list
    # steady_states and epsilon is a uniformly distributed variable that is 
    # drawn from a uniform distribution with variance determined by the input
    # sigma. 
    
    # Create a dofmap
    dofmap = H.dofmap()
    
    # Classify the dofs as either in the hole or on the rest of the sphere
    sphere_dofs = []
    
    # Loop over the cells in the mesh and add the dofs in the respective list
    for cell in cells(mesh):
        # Add all the dofs in the sphere
        sphere_dofs.extend(dofmap.cell_dofs(cell.index()))            
    # Find the unique dofs
    sphere_dofs = list(set(sphere_dofs))
    
    # Access the component steady states as well
    u0 = steady_states[0]
    v0 = steady_states[1]    
    
    # Define an expression for the initial conditions based on the steady states
    if ICs_around_steady_states:
        ic_u = Expression("ic_u",ic_u=u0,degree=1)
        ic_v = Expression("ic_v",ic_v=v0,degree=1)
    else:
        ic_u = Expression("ic_u",ic_u=0,degree=1)
        ic_v = Expression("ic_v",ic_v=0,degree=1)
        
    # Interpolate onto the function space
    ss_u = interpolate(ic_u,H)
    ss_v = interpolate(ic_v,H)
    # Define a function for the random perturbations
    epsilon_u = Function(H)
    epsilon_v = Function(H)
    
    # Fill this vector with a bunch of errors drawn
    # from a standard normal distribution with variance sigma
    epsilon_u = perturbations_ic__Schnakenberg_sphere_with_holes(epsilon_u,sigma)
    epsilon_v = perturbations_ic__Schnakenberg_sphere_with_holes(epsilon_v,sigma)
    
    # Assign the value to our input vector U which will correspond to the initial conditions
    u.vector()[sphere_dofs]=ss_u.vector()[sphere_dofs]+epsilon_u.vector()[sphere_dofs]
    v.vector()[sphere_dofs]=ss_v.vector()[sphere_dofs]+epsilon_v.vector()[sphere_dofs]
    


In [ ]:
# Function 5: "VF_and_FEM_Schnakenberg_sphere_with_holes"

# The function calculates the matrices in the FEM formulation by
# defining the variational formulation and projecting it onto the
# space of continuous picewise linear bases functions over the mesh.
# The function takes the following inputs:
# 1. The list parameters containing the parameters of the Schnakenberg model,
# 2. u and v being the states of the Schnakenberg model,
# 3. phi_1 and phi_2 being the trial functions,
# 4. u_prev, v_prev being the solution at the previous time step,
# 5. dx_list containing the integration measures needed in order to define the variational formulation,
# 6. mesh containing the FEM-mesh of the sphere with a hole (potentially).

# The function returns the following output:
# 1. The mass_matrix in the LHS of the matrix equation resulting from FEM,
# 2.The stiffness_matrix in the LHS of the matrix equation resulting from FEM,
# 3.The reaction_matrix in the LHS of the matrix equation resulting from FEM,
# 4. The mass_load_vector_rhs in the RHS of the matrix equation resulting from FEM,
# 5.The stiffness_load_vector_rhs in the RHS of the matrix equation resulting from FEM,
# 6.The reaction_load_vector_rhs in the RHS of the matrix equation resulting from FEM.
#------------------------------------------------------------------

def VF_and_FEM_Schnakenberg_sphere_with_holes(parameters, u, v, phi_1, phi_2, u_prev, v_prev, dx_list, mesh):
    # Extract the parameters of the Schnakenberg model
    a = parameters[0]
    b = parameters[1]
    d = parameters[2]    
    gamma = parameters[3]
    
    
    # For Schnakenberg kinetics:
    
    # DEFINE THE REACTION TERMS IN THE SCHNAKENBERG MODEL IN A MIXED IMPLICIT EXPLICIT FASHION
    # (here we exclude the gamma factor as we include it later in the time stepping)
    # DISCRETISATION BASED ON THE METHOD CALLED 1-SBEM
    f = a - u + u*u_prev*v_prev
    g = b - u_prev*u_prev*v
    

    
    # DEFINE OUR THREE TYPE OF TERMS IN THE VARIATIONAL FORMULATION (VF):
   
    # NOTE THAT ALL TERMS SHOULD BE CONSIDERED AS BEING ON THE LHS, i.e., LHS=0
    # 1. Mass_form: originating from the time derivatives in the PDEs
    # 2. Stiffness form: originating from the Laplacian in the PDEs
    # 3. Reaction form: originating from the reaction terms in the PDEs
    
    #--------------------------------------------------------
    
    # THE REST OF THE SPHERE
    
    # Extract our first measure for the rest of the sphere
    dx = dx_list[0]
    # Initiate our three forms
    # Time evolution
    mass_form_u = (u-u_prev)*phi_1*dx
    mass_form_v = (v-v_prev)*phi_2*dx
    
    # Diffusion
    stiffness_form_u = dot(grad(u), grad(phi_1))*dx
    stiffness_form_v = d*dot(grad(v), grad(phi_2))*dx
    
    # Reactions
    reaction_form_u = -f*phi_1*dx
    reaction_form_v = -g*phi_2*dx
    
    # Return the output
    return mass_form_u, mass_form_v, stiffness_form_u, stiffness_form_v, reaction_form_u, reaction_form_v 



In [ ]:
# Function 6: "residual_Schnakenberg_sphere_with_holes"

# The function calculates the residual forms being a measure of
# how much the states or concentration profiles change over the course
# of one time step. This is used in order to guide the choice of the
# next time step in the adaptive time stepping procedure where a large
# change in the concentration profile will lead to a small time step
# being chosen and vice versa a small change in the concentration
# profile will lead to a large time step being chosen.

# The input of the function is the following:
# 1. u and v being the states or concentration profiles at the current time step,
# 2. phi_1 and phi_2 being the test functions,
# 3. U_prev being the concentration profiles at the previous time step,
# 4. dx_list being the list of all integration measures.

# The function returns the residual forms.
#------------------------------------------------------------------

def residual_Schnakenberg_sphere_with_holes(parameters,phi_1,phi_2,u_prev,v_prev,u_curr,v_curr,dx_list,mesh,k):
    # Extract the parameters of the Schackenberg model
    a = parameters[0]
    b = parameters[1]
    d = parameters[2]    
    gamma = parameters[3]
    
    # DEFINE THE REACTION TERMS IN THE SCHNAKENBERG MODEL IN A MIXED IMPLICIT EXPLICIT FASHION
    # (here we exclude the gamma factor as we include it later in the time stepping)
    f = a - u_curr + u_curr*u_prev*v_prev
    g = b - u_prev*u_prev*v_curr
    
    # DEFINE OUR THREE TYPE OF TERMS IN THE VARIATIONAL FORMULATION (VF):
    # NOTE THAT ALL TERMS SHOULD BE CONSIDERED AS BEING ON THE LHS, i.e., LHS=0
    # 1. Mass_form: originating from the time derivatives in the PDEs
    # 2. Stiffness form: originating from the Laplacian in the PDEs
    # 3. Reaction form: originating from the reaction terms in the PDEs
    # THE REST OF THE SPHERE
    # Extract our first measure for the rest of the sphere
    dx = dx_list[0]
    
    # Initiate our three forms
    # Time evolution
    mass_form_u = (u_curr-u_prev)*phi_1*dx
    mass_form_v = (v_curr-v_prev)*phi_2*dx
    
    # Diffusion
    stiffness_form_u = dot(grad(u_curr), grad(phi_1))*dx
    stiffness_form_v = d*dot(grad(v_curr), grad(phi_2))*dx
    
    # Reactions
    reaction_form_u = -f*phi_1*dx
    reaction_form_v = -g*phi_2*dx
    
    # Lastly define the residual as well
    residual_form_u = mass_form_u + k*(stiffness_form_u + gamma*reaction_form_u)
    residual_form_v = mass_form_v + k*(stiffness_form_v + gamma*reaction_form_v)    
    
    # Return the residual form
    return residual_form_u + residual_form_v


In [ ]:
# Function 7: "compute_spectral_coefficients_nohole"

def compute_spectral_coefficients_nohole(u, dx, hole_radius,folder_str):
    # Define the radius and the degree of the local approximation
    r = 1.0     # Radius of the sphere
    degree = 1  # Degree of local approximation
    
    
    # DEFINE THE BASIS FUNCTIONS FOR THE SPHERICAL HARMONICS
    # The real spherical harmonics in Cartesian coordinates 
    
    # n=0
    Y_00 = Expression("sqrt(1/(4*pi))", degree=degree)
    # n=1
    Y_10 = Expression("sqrt(3/(4*pi))*x[2]/r", r=r, degree=degree)
    Y_1p1 = Expression("sqrt(3/(4*pi))*x[0]/r", r=r, degree=degree)
    # n=2
    Y_20 = Expression("sqrt(5/(16*pi))*(3*pow(x[2], 2) - pow(r, 2))/pow(r, 2)", r=r, degree=degree)
    Y_2p1 = Expression("sqrt(15/(4*pi))*x[0]*x[2]/pow(r, 2)", r=r, degree=degree)
    Y_2p2 = Expression("sqrt(15/(16*pi))*(pow(x[0], 2) - pow(x[1], 2))/pow(r, 2)", r=r, degree=degree)
    # n=3
    Y_30 = Expression("sqrt(7/(16*pi))*(5*pow(x[2], 3) - 3*x[2]*pow(r, 2))/pow(r, 3)", r=r, degree=degree)
    Y_3p1 = Expression("sqrt(21/(32*pi))*x[0]*(5*pow(x[2], 2) - pow(r, 2))/pow(r, 3)", r=r, degree=degree)
    Y_3p2 = Expression("sqrt(105/(16*pi))*x[2]*(pow(x[0], 2) - pow(x[1], 2))/pow(r, 3)", r=r, degree=degree)
    Y_3p3 = Expression("sqrt(35/(32*pi))*x[0]*(pow(x[0], 2) - 3*pow(x[1], 2))/pow(r, 3)", r=r, degree=degree)
    # n=4
    Y_40 = Expression("sqrt(9/(256*pi))*(35*pow(x[2], 4) - 30*pow(x[2], 2)*pow(r, 2) + 3*pow(r, 4))/pow(r, 4)", r=r, degree=degree)
    Y_4p1 = Expression("sqrt(45/(32*pi))*x[0]*(7*pow(x[2], 3) - 3*x[2]*pow(r, 2))/pow(r, 4)", r=r, degree=degree)
    Y_4p2 = Expression("sqrt(45/(64*pi))*(pow(x[0], 2) - pow(x[1], 2))*(7*pow(x[2], 2) - pow(r, 2))/pow(r, 4)", r=r, degree=degree)
    Y_4p3 = Expression("sqrt(315/(32*pi))*x[0]*x[2]*(pow(x[0], 2) - 3*pow(x[1], 2))/pow(r, 4)", r=r, degree=degree)
    Y_4p4 = Expression("sqrt(315/(256*pi))*(pow(x[0], 2)*(pow(x[0], 2) - 3*pow(x[1], 2)) - pow(x[1], 2)*(3*pow(x[0], 2) - pow(x[1], 2)))/pow(r, 4)", r=r, degree=degree)
    # n=5
    Y_50 = Expression("sqrt(11/(256*pi))*(63*pow(x[2], 5) - 70*pow(x[2],3)*pow(r, 2) + 15*x[2]*pow(r, 4))/pow(r, 5)", r=r, degree=degree)
    Y_5p1 = Expression("sqrt(165/(256*pi))*x[0]*(21*pow(x[2], 4) - 14*pow(x[2], 2)*pow(r, 2) + pow(r, 4))/pow(r, 5)", r=r, degree=degree)
    Y_5p2 = Expression("sqrt(1155/(64*pi))*(pow(x[0], 2) - pow(x[1], 2))*(3*pow(x[2], 3) - x[2]*pow(r, 2))/pow(r, 5)", r=r, degree=degree)
    Y_5p3 = Expression("sqrt(385/(512*pi))*(pow(x[0], 3) - 3*x[0]*pow(x[1], 2))*(9*pow(x[2], 2) - pow(r, 2))/pow(r, 5)", r=r, degree=degree)
    Y_5p4 = Expression("sqrt(3465/(256*pi))*(pow(x[0], 4) - 6*pow(x[0], 2)*pow(x[1], 2) + pow(x[1], 4))*x[2]/pow(r, 5)", r=r, degree=degree)
    Y_5p5 = Expression("sqrt(693/(512*pi))*(pow(x[0], 5) - 10*pow(x[0], 3)*pow(x[1], 2) + 5*x[0]*pow(x[1], 4))/pow(r, 5)", r=r, degree=degree)
    
    
    
    # ASSEMBLE THE SOLUTION AS A FUNCTION OF THESE BASIS FUNCTIONS
    # The real part of the complex spectral coefficients of
    # the supplied function according to Chaplain,
    # i.e., <u, Re(Y_n^m)> = 1/sqrt(2)<u, Y_(n|m|)> for m non-zero
    # and   <u, Re(Y_n^0)> = <u, Y_(n0)> for m = 0, where
    # Y_n^m is a complex spherical harmonic, Y_(n|m|) is a real one
    # and the Condon-Shortley factor has been neglected  
    
    # n=0
    U_00 = assemble(u*Y_00*dx)
    # n=1
    U_10 = assemble(u*Y_10*dx)
    U_1p1 = 1/sqrt(2)*assemble(u*Y_1p1*dx)
    # n=2
    U_20 = assemble(u*Y_20*dx)
    U_2p1 = 1/sqrt(2)*assemble(u*Y_2p1*dx)
    U_2p2 = 1/sqrt(2)*assemble(u*Y_2p2*dx)
    # n=3
    U_30 = assemble(u*Y_30*dx)
    U_3p1 = 1/sqrt(2)*assemble(u*Y_3p1*dx)
    U_3p2 = 1/sqrt(2)*assemble(u*Y_3p2*dx)
    U_3p3 = 1/sqrt(2)*assemble(u*Y_3p3*dx)
    # n=4
    U_40 = assemble(u*Y_40*dx)
    U_4p1 = 1/sqrt(2)*assemble(u*Y_4p1*dx)
    U_4p2 = 1/sqrt(2)*assemble(u*Y_4p2*dx)
    U_4p3 = 1/sqrt(2)*assemble(u*Y_4p3*dx)
    U_4p4 = 1/sqrt(2)*assemble(u*Y_4p4*dx)
    # n=5
    U_50 = assemble(u*Y_50*dx)
    U_5p1 = 1/sqrt(2)*assemble(u*Y_5p1*dx)
    U_5p2 = 1/sqrt(2)*assemble(u*Y_5p2*dx)
    U_5p3 = 1/sqrt(2)*assemble(u*Y_5p3*dx)
    U_5p4 = 1/sqrt(2)*assemble(u*Y_5p4*dx)
    U_5p5 = 1/sqrt(2)*assemble(u*Y_5p5*dx)    
    
    
    # Save the results
    # Create a list of the coefficients of the basis functions
    coeff_list = [U_00, U_10, U_1p1, U_20, U_2p1, U_2p2, U_30, U_3p1, U_3p2, U_3p3, U_40, U_4p1, U_4p2, U_4p3, U_4p4, U_50, U_5p1, U_5p2, U_5p3, U_5p4, U_5p5]
    # Create a list of the corresponding hole radius
    hole_radii_list = [hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius, hole_radius]
    
    
    
    
    # Create an array with the temporary data
    temp_data = np.array([hole_radii_list,coeff_list])
    
    # Create a list of the column names
    col_names = ["Hole radius in the mesh", "Spectral coefficients of the eigenfunctions at final time t=T"]
    
    # Create a list of the names of the eigenfunctions
    row_names = ["$\\gamma_{0,0}$", "$\\gamma_{1,0}$", "$\\gamma_{1,1}$", "$\\gamma_{2,0}$", "$\\gamma_{2,1}$", "$\\gamma_{2,2}$", "$\\gamma_{3,0}$", "$\\gamma_{3,1}$", "$\\gamma_{3,2}$", "$\\gamma_{3,3}$", "$\\gamma_{4,0}$", "$\\gamma_{4,1}$", "$\\gamma_{4,2}$", "$\\gamma_{4,3}$", "$\\gamma_{4,4}$", "$\\gamma_{5,0}$", "$\\gamma_{5,1}$", "$\\gamma_{5,2}$", "$\\gamma_{5,3}$", "$\\gamma_{5,4}$", "$\\gamma_{5,5}$"]
    
    # Create a dataframe
    data = temp_data.T
    
    # Create a pandas data frame which we want to save
    df = pd.DataFrame(data,columns=col_names)
    
    # Add the row_name afterwards
    df.insert(0,'Eigenfunctions', row_names)
    
    # Finally, save the data frame to the given file name
    df.to_csv(folder_str + "/spectral_coefficients.csv")    

    #------------------------------------------------------------------




In [ ]:
# Function 8: "save_IC"

# The function saves the initial condition for the current mesh and parameters. It takes the same input as
# the subsequent function, so read the comments for that function if you wish to know the name of these inputs.



def save_IC(num_holes,steady_states,numerical_parameters,radii_holes,ICs_around_steady_states):
    
    #--------------------------------------------------------------
    # STEP 1 OUT OF 7: EXTRACT PARAMETERS
    #--------------------------------------------------------------    
    
    # Extract the numerical parameters
    sigma = numerical_parameters[0] # Determining the perturbation in the initial conditions
    T = numerical_parameters[1] # Determining the end time for the FD time stepping scheme
    
    #--------------------------------------------------------------
    # STEP 2 OUT OF 7: DEFINE MESHES, INTEGRATION MEASURES AND
    # DEFINE THE HILBERT SPACE FOR THE FEM FORMULATION
    #--------------------------------------------------------------    
    
    # Read in the mesh depending on the number of holes on the sphere
    mesh, mvc_subdomains, mf_subdomains, dx_list = read_mesh_Schnakenberg_sphere_with_holes(num_holes,radii_holes)
    # Define the finite element space for the Schackenberg model using the mesh
    H = FunctionSpace(mesh, "P", 1)
    
    #--------------------------------------------------------------
    # STEP 3 OUT OF 7: DEFINE TEST FUNCTIONS AND TRIAL FUNCTIONS (I.E.
    # THE SOLUTION TO THE SCHNAKENBERG MODEL)
    #--------------------------------------------------------------    
    # Define the previous time step as a function of the function space H
    u_prev = Function(H) # Previous time step in the FD time stepping scheme
    v_prev = Function(H) # Previous time step in the FD time stepping scheme
    
    #--------------------------------------------------------------
    # STEP 4 OUT OF 7: SET THE INITIAL CONDITIONS OF THE SYSTEM,
    # CREATE AN OUTPUT FOLDER WHERE THE RESULTS ARE STORED
    # AND SAVE THE INITIAL CONDITIONS.
    #--------------------------------------------------------------    
    
    # Calculate the initial conditions
    initial_conditions_Schnakenberg_sphere_with_holes(H,mesh,mf_subdomains,num_holes,steady_states,sigma,u_prev,v_prev,ICs_around_steady_states)
    
    # Save the initial conditions to files
    File('Output/fixed_IC_u.xml') << u_prev
    File('Output/fixed_IC_v.xml') << v_prev
    # Save the mesh to a file as well
    File('Output/fixed_IC_mesh.xml') << mesh

    #------------------------------------------------------------------







In [ ]:
# Function 9: "solve_RD_system"
#------------------------------------------------------------------
def solve_RD_system(repitition_index,parameters,numerical_parameters,load_IC,radii_holes,num_holes,steady_states,ICs_around_steady_states):
    
    #--------------------------------------------------------------
    # STEP 1 OUT OF 7: EXTRACT PARAMETERS
    #--------------------------------------------------------------    
    
    # Extract the parameters of the Schnakenberg model
    a = parameters[0]
    b = parameters[1]
    d = parameters[2]    
    gamma = parameters[3]
    n = parameters[4]
    
    # Extract the numerical parameters
    sigma = numerical_parameters[0] # Determining the perturbation in the initial conditions
    T = numerical_parameters[1] # Determining the end time for the FD time stepping scheme
    
    # Re-define parameters as constants
    a = Constant(a)
    b = Constant(b)
    d = Constant(d)
    gamma = gamma
    gamma_const = Constant(gamma)
    
    # Save these in a new list
    parameters_as_constants = [a, b, d, gamma]    
    
    #--------------------------------------------------------------
    # STEP 2 OUT OF 7: DEFINE MESHES, INTEGRATION MEASURES AND
    # DEFINE THE HILBERT SPACE FOR THE FEM FORMULATION
    #--------------------------------------------------------------  
    
    # Read in the mesh depending on the number of holes on the sphere
    mesh, mvc_subdomains, mf_subdomains, dx_list = read_mesh_Schnakenberg_sphere_with_holes(num_holes,radii_holes)
    # Define the finite element space for the Schackenberg model using the mesh
    H = FunctionSpace(mesh, "P", 1)
    
    #--------------------------------------------------------------
    # STEP 3 OUT OF 7: DEFINE TEST FUNCTIONS AND TRIAL FUNCTIONS (I.E.
    # THE SOLUTION TO THE SCHNAKENBERG MODEL)
    #--------------------------------------------------------------    
    
    # Define test functions for the variational formulation (VF) 
    phi_1 = TestFunction(H)
    phi_2 = TestFunction(H)
    
    # Define the trial functions (i.e. solutions of the Schnakenberg model) for the VF.
    # Think of them as the "analytical solution" which we want to approximate.
    u = TrialFunction(H)
    v = TrialFunction(H)
    
    # Define the previous time step as a function of the function space H
    u_prev = Function(H) # Previous time step in the FD time stepping scheme
    u_curr = Function(H) # Current time step in the FD time stepping scheme
    v_prev = Function(H) # Previous time step in the FD time stepping scheme
    v_curr = Function(H) # Current time step in the FD time stepping scheme
    
    #--------------------------------------------------------------
    # STEP 4 OUT OF 7: SET THE INITIAL CONDITIONS OF THE SYSTEM,
    # CREATE AN OUTPUT FOLDER WHERE THE RESULTS ARE STORED
    # AND SAVE THE INITIAL CONDITIONS.
    #--------------------------------------------------------------    
    
    # We wish to have the most explanatory name of our output folder explaining the chosen parameters. So we create a 
    # series of strings which gives us all the information we need in the very name of the folder
    folder_str = "Output/"
    hole_str = "h_" + str(num_holes) + "_"    
    radius_str = ""
    
    for radius in radii_holes:
        radius_str += "r_" + str(round(radius,3)).replace(".","p") + "_"
    a_str = "a_" + str(round(a,3)).replace(".","p") + "_"
    b_str = "b_" + str(round(b,3)).replace(".","p") + "_"
    d_str = "d_" + str(round(d,3)).replace(".","p") + "_"
    gamma_str = "gamma_" + str(round(gamma,3)).replace(".","p") + "_"
    sigma_str = "sigma_" + str(round(sigma,5)).replace(".","p") + "_"
    T_str = "T_" + str(round(T,3)).replace(".","p") + "_"
    
    if ICs_around_steady_states:
        IC_str = "ICs_around_steady_states/"
    else:
        IC_str = "ICs_at_zero/"
    
    # Gather all these substrings into one giant string where we will save the output files
    output_folder_str = folder_str + hole_str + radius_str + a_str + b_str + d_str + gamma_str + sigma_str + T_str + IC_str
    
    # Define two output files based on this giant result folder where we have one output file for each of the two states
    vtkfile_u = File(output_folder_str+ "iteration_" + str(repitition_index) + "/" + "u.pvd")
    vtkfile_v = File(output_folder_str+"iteration_" + str(repitition_index) + "/" + "v.pvd")        
    
    # Set the time to zero as we are looking at the initial conditions
    t = 0.0
    
    
    if load_IC:
        
        # Load the old mesh
        mesh_old = Mesh('Output/fixed_IC_mesh.xml')
        # Define a function space on the old mesh
        H_old = FunctionSpace(mesh_old, "P", 1)
        # Load the old initial conditions on the old function space
        u_old = Function(H_old, 'Output/fixed_IC_u.xml')
        v_old = Function(H_old, 'Output/fixed_IC_v.xml')
        
        # Interpolate the old initial conditions onto the new function space
        # Note that we need to use LagrangeInterpolator in order for the interpolation
        # to work in parallel
        u_old.set_allow_extrapolation(True)
        LagrangeInterpolator.interpolate(u_prev, u_old)
        v_old.set_allow_extrapolation(True)
        LagrangeInterpolator.interpolate(v_prev, v_old)
    else:
        # Calculate the initial conditions
        initial_conditions_Schnakenberg_sphere_with_holes(H,mesh,mf_subdomains,num_holes,steady_states,sigma,u_prev,v_prev,ICs_around_steady_states)
    
    # Save the two initial conditions in the output folder
    u_prev.rename("Concentration profile, $u(\mathbf{x},t)$","u")
    vtkfile_u << (u_prev, t)
    v_prev.rename("Concentration profile, $v(\mathbf{x},t)$","v")    
    vtkfile_v << (v_prev, t)
    
    #--------------------------------------------------------------
    # STEP 5 OUT OF 7: DEFINE THE MATRICES RESULTING FROM THE
    # VF AND THE FEM
    #--------------------------------------------------------------
    
    # Compute the forms from the VF
    mass_form_u, mass_form_v, stiffness_form_u, stiffness_form_v, reaction_form_u, reaction_form_v = VF_and_FEM_Schnakenberg_sphere_with_holes(parameters, u, v, phi_1, phi_2, u_prev, v_prev, dx_list, mesh)
    
    # Assemble time-independent matrices for LHS
    mass_matrix_u = assemble(lhs(mass_form_u), keep_diagonal=True)
    mass_matrix_v = assemble(lhs(mass_form_v), keep_diagonal=True)
    stiffness_matrix_u = assemble(lhs(stiffness_form_u), keep_diagonal=True)
    stiffness_matrix_v = assemble(lhs(stiffness_form_v), keep_diagonal=True)
    
    # Assemble time-independent vectors for RHS
    reaction_vector_u = assemble(rhs(reaction_form_u))
    reaction_vector_v = assemble(rhs(reaction_form_v))
    
    # Compute time-dependent forms for LHS
    reaction_form_u_lhs = lhs(reaction_form_u)
    reaction_form_v_lhs = lhs(reaction_form_v)
    
    # Compute time-dependent forms for RHS
    mass_form_u_rhs = rhs(mass_form_u)
    mass_form_v_rhs = rhs(mass_form_v)
    
    #--------------------------------------------------------------
    # STEP 6 OUT OF 7: TIME STEPPING USING FD IN TIME AND FEM IN SPACE
    #--------------------------------------------------------------
    
    # Define the constant time step (this you need to calibrate...).
    if n<3: # For n=1,2 a step size of dt=0.01 works
        dt = 1e-2
    elif n<5: # For n=3,4 a step size of dt=0.005 works
        dt = 5e-3 
    elif n==5: # For n=5 a step size of dt=0.001 works
        dt = 1e-3   
        
        
        ## Try and see if these work:
    elif n==6: 
        dt = 1e-3  ## Works
        ## 
#     elif n==7: 
#         dt = 1e-3 
#       # How to define timestep for n undefined...?  
        
    # Re-define this as a constant for the FEM solver    
    k = Constant(dt) # For the fem solver as well
    # Define an iterator for the time stepping keeping track of
    # how many iterations that has passed
    t_it = 0
    # Also, define the current time outside the loop, so that we can save
    # the final concentration profile after the looping is done.
    t = 0
    # Previous time step
    t_prev = 0
    

    # Save the concentration profile when the time has increased with a value 0.5
    save_iteration = 0.5
    
    
    #--------------------------------------------------------------
    # STEP 7 OUT OF 7: CALCULATE THE RESIDUAL FORMS NEEDED FOR THE
    #ADAPTIVE TIME STEPPING
    
    residual_form = residual_Schnakenberg_sphere_with_holes(parameters_as_constants, phi_1, phi_2, u_prev, v_prev, u_curr, v_curr, dx_list, mesh, k)
    #----------------------------------------------------------------------------------
    
   
    # We solve the time stepping adaptively until the end time is reached 
    while t < T:
        
        # Save the previous time step
        t_prev = t
        # Update current time and iteration number 
        t_it += 1
        t += dt
        k = Constant(dt)
        
        # Assemble time-dependent matrices for LHS
        reaction_matrix_u = assemble(reaction_form_u_lhs, keep_diagonal=True)
        reaction_matrix_v = assemble(reaction_form_v_lhs, keep_diagonal=True)
        
        # Assemble time-dependent vectors for RHS
        mass_vector_u = assemble(mass_form_u_rhs)
        mass_vector_v = assemble(mass_form_v_rhs)            
        
        # Assemble system matrices and rhs vector
        # SYSTEM WITH REACTIONS
        A_u = mass_matrix_u + k*(stiffness_matrix_u + gamma*reaction_matrix_u)
        A_v = mass_matrix_v + k*(stiffness_matrix_v + gamma*reaction_matrix_v)
        b_u = mass_vector_u + k*gamma*reaction_vector_u
        b_v = mass_vector_v + k*gamma*reaction_vector_v
        
        # Solve linear variational problems for time step
        solve(A_u, u_curr.vector(), b_u)
        solve(A_v, v_curr.vector(), b_v)
        
    
        # Save and check the solution (every whatever iteration)
        if  (t_prev < save_iteration) and (t > save_iteration):
            # Increase the iterations
            save_iteration += 0.5
            
      
 # ----------------------------------------------------------------- #    
    
#             # UNCOMMENT THIS IF YOU WANT TO SAVE FILES MORE FREQUENTLY.
#             # CURRENTLY, WE ARE ONLY SAVING THE INITIAL CONDITION OF THE
#             # ACTIVE COMPONENT AND THE CONCENTRATION PROFILE OF THE ACTIVE
#             # COMPONENT AT THE FINAL TIME AT t=50

#             # Save the components in the data files
#             u_curr.rename("Concentration profile, $u(\mathbf{x},t)$","u")
#             vtkfile_u << (u_curr, t)
#             v_curr.rename("Concentration profile, $v(\mathbf{x},t)$","v")
#             vtkfile_v << (v_curr, t)
         
 # ----------------------------------------------------------------- #
    
            # Iteration health check
            R = assemble(residual_form)
            l2_norm_R = norm(R, 'l2')
            print("\t\tIteration %d, t\t=\t%0.15f out of %0.3f"%(t_it,t,T))
            print("\t\tl2_norm_of_R = ", l2_norm_R)
            print("\t\tdt = ", dt)
            # In case we have negative concentrations, we break
            if u_curr.vector().min() < 0.0 or v_curr.vector().min() < 0.0:
                print("\n\t\tINSTABILITY DETECTED! ### TERMINATE SIMULATION ###\n")
                break
                
                
        # Update old solution
        u_prev.assign(u_curr)
        v_prev.assign(v_curr)
  

# WE ALSO SAVE THE VERY LAST ITERATION WHEN ALL THE TIME STEPPING IS DONE.
   
    u_curr.rename("Concentration profile, $u(\mathbf{x},t)$","u")
    vtkfile_u << (u_curr, t)
    v_curr.rename("Concentration profile, $v(\mathbf{x},t)$","v")
    vtkfile_v << (v_curr, t)        
    print("\n\n\t\tALL IS FINE AND DANDY HERE!\n\n")
    print("Iterations are finished!")   
    
    
    # Compute and save the spectral coefficients as well
    if len(radii_holes)>0:
        compute_spectral_coefficients_nohole(u_curr, dx_list[0], radii_holes[0],output_folder_str+ "iteration_" + str(repitition_index) + "/")
    else:
        compute_spectral_coefficients_nohole(u_curr, dx_list[0], 0,output_folder_str+ "iteration_" + str(repitition_index) + "/")
    
    
    # Lastly, we want to save the final concentration profile as xml-files
    # Save the initial conditions to files
    File(output_folder_str + "iteration_" + str(repitition_index) + "/final_timestep_u.xml") << u_curr
    File(output_folder_str + "iteration_" + str(repitition_index) + "/final_timestep_v.xml") << v_curr
    # Save the mesh to a file as well
    File(output_folder_str + "iteration_" + str(repitition_index) + "/final_timestep_mesh.xml") << mesh    


In [ ]:
 # Function 10: "FEMFD_simulation_Schnakenberg_sphere_with_holes"

# The functions solves the Schnakenberg RD model on the sphere with potential holes and potentially the parameters are altered in the regions adjacent to the holes. The function does not return any output but it writes the concentration profiles of u and v respectively to vtk files which are stored in an appropriately named sub folder of the folder named "../Output". The function takes the following inputs:
# 1. The parameter num_holes determining which mesh that is read as they are classified according to the number of holes that are added on the sphere,
# 2. The list parameters=[a,b,d,gamma] containing all the parameters of the Schnakenberg model,
# 3. The list steady_states=[u0,v0] containing the two states of the Schnakenberg model,
# 4. The list numerical_parameters=[sigma,T] where sigma determines the perturbation in the initial condition and T determines the end time for the FD time stepping scheme,
# 5. The list radii_holes containing the list of the radii of the holes in the mesh,
# 6. A logical variable called ICs_around_steady_states which determines whether the initial conditions are set to the steady states values or not,
# 7. A logical variable called load_IC which determines whether we generate new ICs or if load some fixed ICs,

def FEMFD_simulation_Schnakenberg_sphere_with_holes(num_holes,parameters,steady_states,numerical_parameters,radii_holes,ICs_around_steady_states,number_of_repititions,load_IC,start_repitition):
    
    # Now, we solve the PDE system, a defined number of repititions. Since each PDE simulations is solved on one core, we do this in parallel!
    
    # Define the pool of workers
    pool = mp.Pool(mp.cpu_count()-1) # Temporary fix so we do not use all cores. We want to be work on the computer while some simulations are running in the background.
    
    # Solve our RD system in parallel
    results = pool.starmap(solve_RD_system,[(repitition_index,parameters,numerical_parameters,load_IC,radii_holes,num_holes,steady_states,ICs_around_steady_states) for repitition_index in range(start_repitition,start_repitition+number_of_repititions)])
    
    # Close the pool
    pool.close()

In [ ]:

# Function 7: "project_eigen_functions_onto_mesh"

def project_eigen_functions_onto_mesh(num_holes,radii_holes):
    
    # Read in the mesh depending on the number of holes on the sphere
    mesh, mvc_subdomains, mf_subdomains, dx_list = read_mesh_Schnakenberg_sphere_with_holes(num_holes,radii_holes)
    
    # Define the finite element space for the Schackenberg model using the mesh
    H = FunctionSpace(mesh, "P", 1)
    
    # Define the radius and the degree of the local approximation
    r = 1.0     # Radius of the sphere
    degree = 1  # Degree of local approximation
    
    # DEFINE THE OUTPUT STRING
    folder_str = "Output/eigenfunctions/"
    hole_str = "h_" + str(num_holes)    
    
    if num_holes == 0:
        radius_str = "/"
    else:
        radius_str = "_"
        for radius in radii_holes:
            radius_str += "r_" + str(round(radius,3)).replace(".","p") + "_/"
    # Gather all these substrings into one giant string where we will save the output files
    output_folder_str = folder_str + hole_str + radius_str
    
    
    
    
    # DEFINE THE BASIS FUNCTIONS FOR THE SPHERICAL HARMONICS
    # The real spherical harmonics in Cartesian coordinates 
    
    # n=0
    Y_00 = Expression("sqrt(1/(4*pi))", degree=degree)
    P_Y_00 = project(Y_00, H)
    vtkfile_00 = File(output_folder_str+ "Y_00.pvd")
    P_Y_00.rename("$Y_{00}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_00")
    vtkfile_00 << (P_Y_00, 0)    
    
    # n=1
    Y_10 = Expression("sqrt(3/(4*pi))*x[2]/r", r=r, degree=degree)
    P_Y_10 = project(Y_10, H)
    vtkfile_10 = File(output_folder_str+ "Y_10.pvd")
    P_Y_10.rename("$Y_{10}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_10")
    vtkfile_10 << (P_Y_10, 0)        
    Y_1p1 = Expression("sqrt(3/(4*pi))*x[0]/r", r=r, degree=degree)
    P_Y_11 = project(Y_1p1, H)
    vtkfile_11 = File(output_folder_str+ "Y_11.pvd")
    P_Y_11.rename("$Y_{11}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_1p1")
    vtkfile_11 << (P_Y_11, 0)            
    
    # n=2
    Y_20 = Expression("sqrt(5/(16*pi))*(3*pow(x[2], 2) - pow(r, 2))/pow(r, 2)", r=r, degree=degree)
    P_Y_20 = project(Y_20, H)
    vtkfile_20 = File(output_folder_str+ "Y_20.pvd")
    P_Y_20.rename("$Y_{20}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_20")
    vtkfile_20 << (P_Y_20, 0)                
    Y_2p1 = Expression("sqrt(15/(4*pi))*x[0]*x[2]/pow(r, 2)", r=r, degree=degree)
    P_Y_21 = project(Y_2p1, H)
    vtkfile_21 = File(output_folder_str+ "Y_21.pvd")
    P_Y_21.rename("$Y_{21}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_2p1")
    vtkfile_21 << (P_Y_21, 0)                
    Y_2p2 = Expression("sqrt(15/(16*pi))*(pow(x[0], 2) - pow(x[1], 2))/pow(r, 2)", r=r, degree=degree)
    P_Y_22 = project(Y_2p2, H)
    vtkfile_22 = File(output_folder_str+ "Y_22.pvd")
    P_Y_22.rename("$Y_{22}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_2p2")
    vtkfile_22 << (P_Y_22, 0)                    
    
    # n=3
    Y_30 = Expression("sqrt(7/(16*pi))*(5*pow(x[2], 3) - 3*x[2]*pow(r, 2))/pow(r, 3)", r=r, degree=degree)
    P_Y_30 = project(Y_30, H)
    vtkfile_30 = File(output_folder_str+ "Y_30.pvd")
    P_Y_30.rename("$Y_{30}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_30")
    vtkfile_30 << (P_Y_30, 0)                
    Y_3p1 = Expression("sqrt(21/(32*pi))*x[0]*(5*pow(x[2], 2) - pow(r, 2))/pow(r, 3)", r=r, degree=degree)
    P_Y_31 = project(Y_3p1, H)
    vtkfile_31 = File(output_folder_str+ "Y_31.pvd")
    P_Y_31.rename("$Y_{31}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_3p1")
    vtkfile_31 << (P_Y_31, 0)                
    Y_3p2 = Expression("sqrt(105/(16*pi))*x[2]*(pow(x[0], 2) - pow(x[1], 2))/pow(r, 3)", r=r, degree=degree)
    P_Y_32 = project(Y_3p2, H)
    vtkfile_32 = File(output_folder_str+ "Y_32.pvd")
    P_Y_32.rename("$Y_{32}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_3p2")
    vtkfile_32 << (P_Y_32, 0)                    
    Y_3p3 = Expression("sqrt(35/(32*pi))*x[0]*(pow(x[0], 2) - 3*pow(x[1], 2))/pow(r, 3)", r=r, degree=degree)
    P_Y_33 = project(Y_3p3, H)
    vtkfile_33 = File(output_folder_str+ "Y_33.pvd")
    P_Y_33.rename("$Y_{33}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_3p3")
    vtkfile_33 << (P_Y_33, 0)                    
    
    # n=4
    Y_40 = Expression("sqrt(9/(256*pi))*(35*pow(x[2], 4) - 30*pow(x[2], 2)*pow(r, 2) + 3*pow(r, 4))/pow(r, 4)", r=r, degree=degree)
    P_Y_40 = project(Y_40, H)
    vtkfile_40 = File(output_folder_str+ "Y_40.pvd")
    P_Y_40.rename("$Y_{40}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_40")
    vtkfile_40 << (P_Y_40, 0)                    
    Y_4p1 = Expression("sqrt(45/(32*pi))*x[0]*(7*pow(x[2], 3) - 3*x[2]*pow(r, 2))/pow(r, 4)", r=r, degree=degree)
    P_Y_41 = project(Y_4p1, H)
    vtkfile_41 = File(output_folder_str+ "Y_41.pvd")
    P_Y_41.rename("$Y_{41}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_4p1")
    vtkfile_41 << (P_Y_41, 0)                    
    Y_4p2 = Expression("sqrt(45/(64*pi))*(pow(x[0], 2) - pow(x[1], 2))*(7*pow(x[2], 2) - pow(r, 2))/pow(r, 4)", r=r, degree=degree)
    P_Y_42 = project(Y_4p2, H)
    vtkfile_42 = File(output_folder_str+ "Y_42.pvd")
    P_Y_42.rename("$Y_{42}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_4p2")
    vtkfile_42 << (P_Y_42, 0)                    
    Y_4p3 = Expression("sqrt(315/(32*pi))*x[0]*x[2]*(pow(x[0], 2) - 3*pow(x[1], 2))/pow(r, 4)", r=r, degree=degree)
    P_Y_43 = project(Y_4p3, H)
    vtkfile_43 = File(output_folder_str+ "Y_43.pvd")
    P_Y_43.rename("$Y_{43}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_4p3")
    vtkfile_43 << (P_Y_43, 0)                    
    Y_4p4 = Expression("sqrt(315/(256*pi))*(pow(x[0], 2)*(pow(x[0], 2) - 3*pow(x[1], 2)) - pow(x[1], 2)*(3*pow(x[0], 2) - pow(x[1], 2)))/pow(r, 4)", r=r, degree=degree)    
    P_Y_44 = project(Y_4p4, H)
    vtkfile_44 = File(output_folder_str+ "Y_44.pvd")
    P_Y_44.rename("$Y_{44}(\mathbf{x}),\;\mathbf{x}\in S^2$","Y_4p4")
    vtkfile_44 << (P_Y_44, 0)                    


### Launch the simulation (note that it depending on what value $n$ is, the simulations can take up to 4+ hours):

- For $n$=1,2 choose final time to be $T$=50, and for $n$=3,4,5,6 choose $T$=20

- Modified to have *no* holes

In [ ]:
# Description:
# This is the script which launches the FEM simulations, and
# it uses all functions that are stored in the script
# "toolbox_FEM_simulations_Schnakenberg_sphere_with_holes.py".

In [ ]:
start_time = time.time()  # Start measuring execution time


# import toolbox_FEM_simulations_Schnakenberg_sphere_with_holes as FEM_toolbox  # Home-made
# import Schnakenberg_properties # Home-made
# =================================================================================
# =================================================================================
# The experiments
# =================================================================================
# =================================================================================

# The parameters in the Schnakenberg model
a = 0.2
b = 1

# Prompt to the user
print("---------------------------------------------------------------------------------------------------------\n")
print("\tRUNNING SCHNAKENBERG'S MODEL ON A SPHERICAL MESH WITH NO HOLE IN IT")
print("---------------------------------------------------------------------------------------------------------\n")


# Set the value of the relative diffusion
d_vec = [18] 

# We have no holes, so no radius necessary
radii_holes = []

# Define the perturbation in the initial conditions
sigma = 1e-4

## ------------------------------------------------------------------------------------------------------------- ##

# Define the end time for the simulations

#T = 500  # for T=50

T = 200   # for T=20

## ------------------------------------------------------------------------------------------------------------- ##


# Collect these latter two parameters in a list as well
numerical_parameters = [sigma, T]

# Looping over the varius radii and run all simulations there!
# Define the experimental design of holes with increasing radii
experimental_design = []



# hole_radius_array = np.arange(0,0.75,0.05) # The full experimental design

## No holes:
hole_radius_array = np.array([0])

## ------------------------------------------------------------------------------------------------------------- ##

# Define the eigenvalues we want to consider 

# (be careful for higher values, need correct dt defined to work)

n_vec = [1] # 1,2,3,4,5,6,... 

## ------------------------------------------------------------------------------------------------------------- ##


# Loop over the eigenvalues

for n_index,n in enumerate(n_vec):
    
    k_squared = n*(n+1)
    
    # Calculate the steady states and the critical parameters
    u_0, v_0, d_c, gamma_c = calculate_steady_states_and_critical_parameters_Schnakenberg(a,b,k_squared)
    
    # Save the steady states in a list
    steady_states = [u_0,v_0]
    
    # Set the value of the relative diffusion to its given value
    d = d_vec[n_index]
    
    # Set the value of the reaction strength to its critical value
    gamma = gamma_c
    # Collect all parameters in a list
    parameters = [a, b, d, gamma, n]
    
    # Check isolated spectral modes for choices of a, b, d, and gamma.
    Turing_conditions,L,M = check_Turing_conditions_Scnakenberg(a,b,d)
    print(gamma*L, gamma*M)
    ninterval = [1, 10]
    compute_isolated_spectral_modes(ninterval, gamma, L, M)
    
    # Print the results
    print("\n\t\tThe steady states:\t\t\t(u_0,v_0)\t=\t(%0.4f,%0.4f)"%(u_0,v_0))
    print("\t\tThe critical parameters:\t\t(d_c,gamma_c)\t=\t(%0.4f,%0.4f)"%(d_c,gamma_c))    
    
    # Loop over the hole_radii and add the experiments
    for hole_index,hole_radius in enumerate(hole_radius_array):
        
        # Special case for the mesh with no hole
        if hole_radius == 0:
            experimental_design.append((0,parameters,steady_states,numerical_parameters,[],True,False))
        else:
            experimental_design.append((1,parameters,steady_states,numerical_parameters,[hole_radius],True,False))

            
# Here, we define the start repititions and the number of repititions
          
#number_of_repititions = 20 # For the full experimental design
number_of_repititions = 1

# This value we can tweek if we want to add extra simulations afterwards
start_repitition = 0 

# Loop over the experiments in the experimental design and run them all (with the appropriate number of repititions)
for experiment in experimental_design:
    
    # Prompt to the user
    print("---------------------------------------------------------------------------------------------------------\n")
    print("\tNUM_HOLES\t=\t%d,\tRADII\t=\t%s"%(int(experiment[0]),str(experiment[4])))
    print("---------------------------------------------------------------------------------------------------------\n")    
    
    # Solve the FEM system with the given parameters
    FEMFD_simulation_Schnakenberg_sphere_with_holes(experiment[0],experiment[1],experiment[2],experiment[3],experiment[4],experiment[5],number_of_repititions,experiment[6],start_repitition)

    
    
    
end_time = time.time()  # End measuring execution time  
execution_time = end_time - start_time  # Calculate execution time
print("Execution Time: %.2f seconds" % execution_time)    

## Some times to expect for different values of n:

     ## Execution Time: ~4126.63 seconds for n=1,2. 
     
     ## n=3: Execution Time: 2908.65 seconds
   
     ## n=4: Execution Time: 3051.10 seconds

     ## n=5: Execution Time: 15560.21 seconds

 


### Plots:

In [ ]:
# Description:
# This script reads the final concentration profile and decomposes it in terms
# of the eigenfunctions of the Laplace Beltrami operator

In [ ]:
# The parameters in the Schnakenberg model
a = 0.2
b = 1.0

## ------------------------------------------------------------------------------------------------------------- ##

# The wavenumber k^2
# n = 1,2,3,4,5...

n = 1

## ------------------------------------------------------------------------------------------------------------- ##


k_squared = n*(n+1)

# Calculate the steady states and the critical parameters
u_0, v_0, d_c, gamma_c = calculate_steady_states_and_critical_parameters_Schnakenberg(a,b,k_squared)

# Save the steady states in a list
steady_states = [u_0,v_0]

# Set the value of the relative diffusion
d = 18 

# Set the value of the reaction strength to its critical value
gamma = gamma_c

# Define the number of holes
num_holes = 0
# Define the radius
radii_holes = []
# Define that we have the ICs around the steady states
ICs_around_steady_states = True
# Define the perturbation in the initial conditions
sigma = 1e-4


# Define the end time for the simulations (500==50, 200==20 etc)

## ------------------------------------------------------------------------------------------------------------- ##

# T = 50
T = 20

## ------------------------------------------------------------------------------------------------------------- ##


# Let's start with the zeroth repitition
repitition_index = 0

# Define the meshes we want to loop over
hole_radius_array = np.array([0])

# Allocate a list of all the basis functions
basis_functions = []

# Let's add 20 lists for each basis function corresponding to
# n = 1,2,3,4,5...
for index in range(21):
    basis_functions.append([])   
    
#----------------------------------------------------------------------------------


#----------------------------------------------------------------------------------

# DEFINE THE FOLDERS WE LOOK THROUGH

## n=1: 
#folder_str = "Output/h_0_a_0p2_b_1p0_d_18p0_gamma_6p873_sigma_0p0001_T_500_ICs_around_steady_states/iteration_0/"

## Note the folder name:

# h_0 ~ no holes
# a_0p2 ~ a=0.2
# b_1p0 ~ b=1.0
# d_18p0 ~ d=18.0
# gamma_6p873 ~ gamma = 6.873
# sigma_0p0001 ~ sigma=0.0001
# T_500 ~ T=500



### ...

## n=3:
#folder_str = "Output/h_0_a_0p2_b_1p0_d_18p0_gamma_41p238_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/"

## n=4: 
# folder_str = "Output/h_0_a_0p2_b_1p0_d_18p0_gamma_68p73_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/"

## n=5: 
#folder_str = "Output/h_0_a_0p2_b_1p0_d_18p0_gamma_103p095_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/"

## n=6: 
#folder_str = "Output/h_0_a_0p2_b_1p0_d_18p0_gamma_144p333_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/"


## ------------------------------------------------------------------------------------------------------------- ##

## Current folder: 
folder_str = "Output/...../iteration_0/"

## ------------------------------------------------------------------------------------------------------------- ##


a_str = "a_" + str(round(a,3)).replace(".","p") + "_"
b_str = "b_" + str(round(b,3)).replace(".","p") + "_"
d_str = "d_" + str(round(d,3)).replace(".","p") + "_"
gamma_str = "gamma_" + str(round(gamma,3)).replace(".","p") + "_"
sigma_str = "sigma_" + str(round(sigma,5)).replace(".","p") + "_"
T_str = "T_" + str(round(T,3)).replace(".","p") + "_"
if ICs_around_steady_states:
    IC_str = "ICs_around_steady_states/"
else:
    IC_str = "ICs_at_zero/"
    
    
    
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------


# Allocate memory for the legend_strings
legend_strings = []

# Add a help variable
help_variable = 0

# Read ALL the data by looping over all hole radii
for hole_index in range(len(hole_radius_array)):
    if hole_index == 0:
        hole_str = "h_0_"
        radii_holes = []
    else:
        hole_str = "h_1_"
        radii_holes = [hole_radius_array[hole_index]]
    # Define an empty hole radius string    
    radius_str = ""
    
    
    # Loop over all raddi and read them one by one
    for radius in radii_holes:
        radius_str += "r_" + str(round(radius,3)).replace(".","p") + "_"
    # Now, the hole and the radii string is finished, extract the basis functions corresponding to the zeroth iteration
    for repitition_index in range(1):
        # Gather all these substrings into one giant string where we will save the output files
        output_folder_str = folder_str + hole_str + radius_str + a_str + b_str + d_str + gamma_str + sigma_str + T_str + IC_str + "iteration_" + str(repitition_index) + "/"
        
        
        
        # Read the csv file
        #dataframe = read_csv(output_folder_str + "spectral_coefficients.csv", header=None)
       
        ## n=1:
        #dataframe = read_csv("Output/h_0_a_0p2_b_1p0_d_20p0_gamma_6p873_sigma_0p0001_T_500_ICs_around_steady_states/iteration_0/spectral_coefficients.csv", header=None)

        ## n=2 
        #dataframe = read_csv("Output/h_0_a_0p2_b_1p0_d_18p0_gamma_20p619_sigma_0p0001_T_500_ICs_around_steady_states/iteration_0/spectral_coefficients.csv", header=None)

        ## n=3
        #dataframe = read_csv("Output/h_0_a_0p2_b_1p0_d_18p0_gamma_41p238_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/spectral_coefficients.csv", header=None)


        ## n=4
        #dataframe = read_csv("Output/h_0_a_0p2_b_1p0_d_18p0_gamma_68p73_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/spectral_coefficients.csv", header=None)

        
        ## n=5
        #dataframe = read_csv("Output/h_0_a_0p2_b_1p0_d_18p0_gamma_103p095_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/spectral_coefficients.csv", header=None)

        ## n=6
        #dataframe = read_csv("Output/h_0_a_0p2_b_1p0_d_18p0_gamma_144p333_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/spectral_coefficients.csv", header=None)

        
## ------------------------------------------------------------------------------------------------------------- ##
        
        ## Current folder: 
        dataframe = read_csv("Output/..../iteration_0/spectral_coefficients.csv", header=None)

## ------------------------------------------------------------------------------------------------------------- ##
        
        # Save the legends one time
        if help_variable == 0:
            
            # Save the legend strings
            legend_strings = list(dataframe.values[:,1])
            # Remove the first value
            del legend_strings[0]
            
            # Loop over all legends and change their name
            for index,legend in enumerate(legend_strings):
                legend_strings[index] = legend_strings[index].replace("\\gamma","U")
            # Increment the legend string so that we do not save it anymore
            help_variable += 1
            
        # Loop through our data frame and save each value (we need to cast it as a double first)
        for index in range(len(dataframe.values[:,3])):
            if index>0:
                # Append the casted value to our data frame
                basis_functions[index-1].append(float(dataframe.values[index,3]))

                
                

                
# Save indices            
basis_functions = np.array([value[0] for value in basis_functions])
basis_function_index = np.array([index for index,value in enumerate(basis_functions)])

#===================================================================================================================================================

# Set all parameters to tex
plt.rcParams['text.usetex'] = False


                 # Plot our lovely solutions


fig_1 = plt.figure(constrained_layout=True, figsize=(20, 8))
# plt.plot(basis_function_index, basis_functions, '*' ,color=(0/256,0/256,0/256),linewidth=3.0)
plt.plot(basis_function_index, basis_functions, '*', color=(0/256,0/256,0/256), linewidth=3.0, label='Spectral coefficients')

plt.grid()
plt.legend(loc='best',prop={"size":20})
plt.xlabel(xlabel='$(n,m)$-indices for $U_{n}^{m}$',fontsize=25)
plt.ylabel(ylabel='Spectrail coefficients $U_{n}^{m}$',fontsize=25)

# Change the size of the ticks
plt.tick_params(axis='both', which='major', labelsize=20)
plt.tick_params(axis='both', which='minor', labelsize=20)


# Title and saving the figure
plt.title('Spectral decomposition of $u(\\mathbf{x},t=20)$',fontsize=30,weight='bold')

# Set the xticklabels
ax = plt.gca()
ax.set_xticks(list(basis_function_index))
ax.set_xticklabels(['$(0,0)$', '$(1,0)$', '$(1,1)$', '$(2,0)$', '$(2,1)$', '$(2,2)$', '$(3,0)$', '$(3,1)$', '$(3,2)$', '$(3,3)$', '$(4,0)$', '$(4,1)$', '$(4,2)$', '$(4,3)$', '$(4,4)$','$(5,0)$', '$(5,1)$', '$(5,2)$', '$(5,3)$', '$(5,4)$', '$(5,5)$'])


# Show the plot
plt.savefig('Figures/spectral_decomposition_no_holes_u_at_time_t_20.png')

#plt.show() # Uncomment if you want a figure to pop-up
#===================================================================================================================================================



### Plot output:

In [ ]:
import pyvista as pv

# Load the .vtu file
# Ufinal = pv.read("Output/h_0_a_0p2_b_1p0_d_18p0_gamma_103p095_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/u000001.vtu")
# Vfinal = pv.read("Output/h_0_a_0p2_b_1p0_d_18p0_gamma_103p095_sigma_0p0001_T_200_ICs_around_steady_states/iteration_0/v000001.vtu")

# Load the .vtu file (final time here were denoted 000001)
Ufinal = pv.read("Output/.../iteration_0/u000001.vtu")
Vfinal = pv.read("Output/.../iteration_0/v000001.vtu")


# Plot the mesh
p = pv.Plotter()
p.add_mesh(Ufinal)
p.show()

q = pv.Plotter()
q.add_mesh(Vfinal)
q.show()